In [1]:
import torch
import os
from tqdm import tqdm
import torch
from datasets import load_from_disk
from transformers import HubertModel, Wav2Vec2FeatureExtractor, RobertaTokenizer, RobertaModel

save_dir = "precomputed_embeddings"
os.makedirs(save_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [2]:
audio_checkpoint = 'facebook/hubert-base-ls960'
text_checkpoint = 'roberta-base'

hubert_model = HubertModel.from_pretrained(audio_checkpoint).to(device)
hubert_processor = Wav2Vec2FeatureExtractor.from_pretrained(audio_checkpoint)

roberta_model = RobertaModel.from_pretrained(text_checkpoint).to(device)
roberta_tokenizer = RobertaTokenizer.from_pretrained(text_checkpoint)


def extract_hubert_features(audio_array):
    audio_tensor = torch.tensor(audio_array).to(device)
    input_values = hubert_processor(audio_tensor, return_tensors="pt", sampling_rate=16000).input_values.to(device)

    with torch.no_grad():
        hubert_features = hubert_model(input_values).last_hidden_state  # (1, seq_len, 768)

    return torch.mean(hubert_features, dim=1).squeeze(0).tolist()


def extract_roberta_features(transcript):
    tokens = roberta_tokenizer(transcript, return_tensors="pt", padding=True, truncation=True, max_length=128)
    tokens = {key: val.to(device) for key, val in tokens.items()}

    with torch.no_grad():
        roberta_output = roberta_model(**tokens)

    return roberta_output.last_hidden_state[:, 0, :].squeeze(0).tolist()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dataset = load_from_disk('./iemocap')

emotion_labels = ['neutral', 'happy', 'sad', 'angry', 'frustrated', 'excited', 'fear', 'disgust', 'surprise', 'other']
num_classes = len(emotion_labels)

label_to_idx = {label: idx for idx, label in enumerate(emotion_labels)}

dataset


DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'frustrated', 'angry', 'sad', 'disgust', 'excited', 'fear', 'neutral', 'surprise', 'happy', 'EmoAct', 'EmoVal', 'EmoDom', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db'],
        num_rows: 10039
    })
})

In [4]:
print(f"HuBERT Model Device: {next(hubert_model.parameters()).device}")
print(f"RoBERTa Model Device: {next(roberta_model.parameters()).device}")
print(f"Expected Device: {device}")

HuBERT Model Device: cuda:0
RoBERTa Model Device: cuda:0
Expected Device: cuda


In [5]:
print("Extracting embeddings")

modified_dataset = dataset.map(
    lambda example: {
        "audio_embedding": extract_hubert_features(example["audio"]["array"]),
        "text_embedding": extract_roberta_features(example["transcription"]),
        "label_id": label_to_idx[example["major_emotion"]]
    },
    batched=False,
    load_from_cache_file=False
)

print("Complete.")

Extracting embeddings


Parameter 'function'=<function <lambda> at 0xff58c1628360> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/10039 [00:00<?, ? examples/s]

Complete.


In [6]:
save_path = "./iemocap_precomputed"
modified_dataset.save_to_disk(save_path)
print(f"Dataset with embeddings saved to {save_path}")

Saving the dataset (0/4 shards):   0%|          | 0/10039 [00:00<?, ? examples/s]

Dataset with embeddings saved to ./iemocap_precomputed


In [7]:
modified_dataset

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'frustrated', 'angry', 'sad', 'disgust', 'excited', 'fear', 'neutral', 'surprise', 'happy', 'EmoAct', 'EmoVal', 'EmoDom', 'gender', 'transcription', 'major_emotion', 'speaking_rate', 'pitch_mean', 'pitch_std', 'rms', 'relative_db', 'audio_embedding', 'text_embedding', 'label_id'],
        num_rows: 10039
    })
})